In [1]:
import tensorflow as tf
from keras.models import Sequential
import keras.layers as layers
from keras.preprocessing.image import ImageDataGenerator
from spp.SpatialPyramidPooling import SpatialPyramidPooling
import os
import random

import numpy as np
from typing import Tuple

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [3]:
# Locations
FOLDER_LOCATION = "E:\Downloads\Cats-vs-Dogs-all"
TEST_FOLDER = "test"
TRAIN_FOLDER = "train"

# Aims
## Keras
I aim to generate a CNN that is capable of identifying the difference between cats and dogs


Layer 1:
CNN
Number of channels: 200
Kernel: [3, 3]
activation: RELU
[2, 2]

Layer 2:
CNN
Number of channels: 200
Kernel: [3, 3]
activation: RELU
[2, 2]

Layer 4:
CNN
Number of channels: 100
Kernel: [3, 3]
activation: RELU
[2, 2]

Potential Global Pooling layer

Layer 5:
FC
nodes: 300
dropout 0.4
activation: RELU

Layer 6:
FC
nodes: 200
dropout: 0.4
activation: RELU

Layer 7:
FC
nodes: 125
dropout: 0.4
activation: RELU

Layer 8:
FC
nodes: 2
activation: Linear

Use Logits to obtain system accuracy


In [4]:
# Load in the training data and shuffle it with KFolds and a validation dataset
BATCH_SIZE = 100
IMAGE_HEIGHT = 300
IMAGE_WIDTH = 300
image_data_generator = ImageDataGenerator(rescale=1./255,
                                          horizontal_flip=True,
                                          rotation_range=50,
                                          validation_split=0.1)
train_generator = image_data_generator.flow_from_directory(os.path.join(FOLDER_LOCATION, TRAIN_FOLDER), classes=['cat', 'dog'], 
                                                           class_mode="categorical", 
                                                           batch_size=BATCH_SIZE, 
                                                           target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), 
                                                           subset="training")

validation_generator = image_data_generator.flow_from_directory(os.path.join(FOLDER_LOCATION, TRAIN_FOLDER), 
                                                                classes=['cat', 'dog'],
                                                                class_mode="categorical", 
                                                                batch_size=BATCH_SIZE, 
                                                                target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), 
                                                                subset="validation")
print("{} training samples".format(train_generator.samples))


Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
22500 training samples


In [5]:
LAYER_1_CNN = 200
LAYER_2_CNN = 100
LAYER_3_CNN = 50

LAYER_1_FC = 100
LAYER_2_FC = 100
LAYER_3_FC = 50
LAYER_OUTPUT = 2

model = Sequential()
model.add(layers.Convolution2D(filters=6, kernel_size=(3,3), strides=(1,1), activation="relu", 
                               input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
model.add(layers.Convolution2D(filters=10, kernel_size=(5,5), strides=(1,1), activation="relu"))
model.add(layers.Convolution2D(filters=6, kernel_size=(3,3), strides=(1,1), activation="relu"))
model.add(layers.Convolution2D(filters=5, kernel_size=(3,3), strides=(1,1), activation="relu"))
model.add(SpatialPyramidPooling([1, 5]))
model.add(layers.Dense(LAYER_1_FC, activation="relu", name="FC_1"))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(LAYER_2_FC, activation="relu", name="FC_2"))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(LAYER_3_FC, activation="relu", name="FC_3"))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(LAYER_OUTPUT, activation="softmax", name="FC_output"))
model.summary()
print(IMAGE_HEIGHT)
print(IMAGE_WIDTH)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 6)       168       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 294, 294, 10)      1510      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 292, 292, 6)       546       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 290, 290, 5)       275       
_________________________________________________________________
spatial_pyramid_pooling_1 (S (None, 130)               0         
_________________________________________________________________
FC_1 (Dense)                 (None, 100)               13100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
__________

In [5]:
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
# Trains for 10 epochs.

model.fit_generator(train_generator, 
                    steps_per_epoch=train_generator.samples//BATCH_SIZE, 
                    epochs=10, 
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.samples//BATCH_SIZE)

Epoch 1/10
225/225 [==============================] - 664s 3s/step - loss: 0.6685 - acc: 0.5825 - val_loss: 0.6135 - val_acc: 0.6684
Epoch 2/10
225/225 [==============================] - 462s 2s/step - loss: 0.6001 - acc: 0.6788 - val_loss: 0.5283 - val_acc: 0.7316
Epoch 3/10
225/225 [==============================] - 462s 2s/step - loss: 0.5602 - acc: 0.7186 - val_loss: 0.5243 - val_acc: 0.7336
Epoch 4/10
225/225 [==============================] - 463s 2s/step - loss: 0.5407 - acc: 0.7338 - val_loss: 0.5081 - val_acc: 0.7476
Epoch 5/10
225/225 [==============================] - 464s 2s/step - loss: 0.5300 - acc: 0.7418 - val_loss: 0.5236 - val_acc: 0.7360
Epoch 6/10
225/225 [==============================] - 460s 2s/step - loss: 0.5209 - acc: 0.7445 - val_loss: 0.4796 - val_acc: 0.7712
Epoch 7/10
225/225 [==============================] - 465s 2s/step - loss: 0.5126 - acc: 0.7517 - val_loss: 0.4692 - val_acc: 0.7836
Epoch 8/10
225/225 [==============================] - 463s 2s/step - 

In [6]:
model.save("model_sixties")

In [6]:
model.load_weights("model_sixties")